In [1]:
import re
import string
import mlflow
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('IMDB.csv')

df = df.sample(500)

df.to_csv('data.csv', index=False)

df.head()

,review,sentiment
764,I'm a large scarred heterosexual male ex-bounc...,positive
64,Wimpy stuffed shirt Armand Louque (blandly pla...,negative
234,This is the only full length feature film abou...,positive
487,I'm sure that any legitimate submariner would ...,negative
555,The name (Frau) of the main character is the G...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)

df.head()

,review,sentiment
764,large scarred heterosexual male ex bouncer ex ...,positive
64,wimpy stuffed shirt armand louque blandly play...,negative
234,full length feature film world bridge found fi...,positive
487,sure legitimate submariner would happily ship ...,negative
555,name frau main character german word woman kno...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])

df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})

df.head()

,review,sentiment
764,large scarred heterosexual male ex bouncer ex ...,1
64,wimpy stuffed shirt armand louque blandly play...,0
234,full length feature film world bridge found fi...,1
487,sure legitimate submariner would happily ship ...,0
555,name frau main character german word woman kno...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)

X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project.mlflow')
dagshub.init(repo_owner='ShwaTech', repo_name='MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/home/mo-shwa/anaconda3/envs/MLOps-env/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=193554a1-6b66-46aa-a602-8d006f66a0e6&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=59701eb1638de3538ccf44217d72e60b433bf88b6374328cfe26bd209d265d77


Opening in existing browser session.


Accessing as ShwaTech

Initialized MLflow to track repo "ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project"

Repository ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project initialized!

2025/10/16 22:55:26 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/42d7ddac9c90425e916f012f1393c3af', creation_time=1760644527035, experiment_id='0', last_update_time=1760644527035, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [24]:
import mlflow
import logging
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.save_model(model, "logistic_regression_model")
        mlflow.log_artifact("logistic_regression_model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-16 23:26:38,831 - INFO - Starting MLflow run...
2025-10-16 23:26:39,244 - INFO - Logging preprocessing parameters...
2025-10-16 23:26:40,513 - INFO - Initializing Logistic Regression model...
2025-10-16 23:26:40,521 - INFO - Fitting the model...
2025-10-16 23:26:40,643 - INFO - Model training complete.
2025-10-16 23:26:40,643 - INFO - Logging model parameters...
2025-10-16 23:26:41,017 - INFO - Making predictions...
2025-10-16 23:26:41,022 - INFO - Calculating evaluation metrics...
2025-10-16 23:26:41,109 - INFO - Logging evaluation metrics...
2025-10-16 23:26:46,169 - INFO - Saving and logging the model...
2025-10-16 23:26:53,827 - INFO - Model training and logging completed in 14.58 seconds.
2025-10-16 23:26:53,828 - INFO - Accuracy: 0.664
2025-10-16 23:26:53,829 - INFO - Precision: 0.6911764705882353
2025-10-16 23:26:53,830 - INFO - Recall: 0.6911764705882353
2025-10-16 23:26:53,830 - INFO - F1 Score: 0.6911764705882353


🏃 View run painted-crane-564 at: https://dagshub.com/ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project.mlflow/#/experiments/0/runs/136e83a11d9f40e29e4923336a426294
🧪 View experiment at: https://dagshub.com/ShwaTech/MLOps-End-To-End-Movie-Review-Sentiment-Analysis-Project.mlflow/#/experiments/0
